In [4]:
import pandas as pd
import datetime as dt
import numpy as np
import requests
import json
import math

In [5]:
df = pd.read_csv("aq_2018_to_2024.csv")

In [6]:
df

,idSensore,Data,Valore,Stato,idOperatore
0,5504,01/01/2018 01:00:00,56.9,VA,1
1,5504,01/01/2018 02:00:00,53.8,VA,1
2,5504,01/01/2018 03:00:00,68.0,VA,1
3,5504,01/01/2018 04:00:00,60.1,VA,1
4,5504,01/01/2018 05:00:00,59.7,VA,1
...,...,...,...,...,...
17298307,30166,31/12/2023 20:00:00,2.4,VA,1
17298308,30166,31/12/2023 21:00:00,2.2,VA,1
17298309,30166,31/12/2023 22:00:00,2.3,VA,1
17298310,30166,31/12/2023 23:00:00,2.4,VA,1


In [7]:
# df = pd.read_csv("aq_sensor_data.csv")
df = df[['idSensore', 'Data', 'Valore']]
df["Data"] = pd.to_datetime(df['Data'], format='%d/%m/%Y %H:%M:%S')
df

/tmp/ipykernel_104513/3097309402.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Data"] = pd.to_datetime(df['Data'], format='%d/%m/%Y %H:%M:%S')


,idSensore,Data,Valore
0,5504,2018-01-01 01:00:00,56.9
1,5504,2018-01-01 02:00:00,53.8
2,5504,2018-01-01 03:00:00,68.0
3,5504,2018-01-01 04:00:00,60.1
4,5504,2018-01-01 05:00:00,59.7
...,...,...,...
17298307,30166,2023-12-31 20:00:00,2.4
17298308,30166,2023-12-31 21:00:00,2.2
17298309,30166,2023-12-31 22:00:00,2.3
17298310,30166,2023-12-31 23:00:00,2.4


In [16]:
#df['Data'] = pd.to_datetime(df['Data'], format="%d/%m/%Y %I:%M:%S %p", errors="coerce") #01/01/2023 12:00:00 AM
df = df.loc[
    (df["Data"] >= "2018-1-1") & (df["Data"] <= "2023-12-31")
]
df

,idSensore,Data,Valore
0,5504,2018-01-01 01:00:00,56.9
1,5504,2018-01-01 02:00:00,53.8
2,5504,2018-01-01 03:00:00,68.0
3,5504,2018-01-01 04:00:00,60.1
4,5504,2018-01-01 05:00:00,59.7
...,...,...,...
17298283,30166,2023-12-30 20:00:00,3.1
17298284,30166,2023-12-30 21:00:00,3.2
17298285,30166,2023-12-30 22:00:00,2.8
17298286,30166,2023-12-30 23:00:00,3.1


In [17]:
complete_air_quality_stations = pd.read_csv("complete_air_quality_stations.csv")

In [18]:
sensors_in_data = list(df.idSensore.unique())
stations_sensors = list(complete_air_quality_stations.IdSensore.unique())

# Nulling stations that do not exist
missing_sensors = []
for data_sensor in sensors_in_data:
    exist = False
    for station_sensor in stations_sensors:
        if station_sensor == data_sensor: exist = True
    if not exist: missing_sensors.append(data_sensor)
    
missing_sensors

[]

In [19]:
# Remove observations of stations that do not exist
for sensor in missing_sensors:
    print(sensor)
    df = df.loc[df["idSensore"] != sensor]

df

,idSensore,Data,Valore
0,5504,2018-01-01 01:00:00,56.9
1,5504,2018-01-01 02:00:00,53.8
2,5504,2018-01-01 03:00:00,68.0
3,5504,2018-01-01 04:00:00,60.1
4,5504,2018-01-01 05:00:00,59.7
...,...,...,...
17298283,30166,2023-12-30 20:00:00,3.1
17298284,30166,2023-12-30 21:00:00,3.2
17298285,30166,2023-12-30 22:00:00,2.8
17298286,30166,2023-12-30 23:00:00,3.1


In [20]:
def arrange_timestamp(date_time):
    return str(date_time)

In [21]:
base_url_measurements = "http://127.0.0.1:8000/api/collections/airqualitymeasurement/"

batch_size = 1000
batches = math.ceil(len(df) / batch_size)
#batches = 1
print(f"Number of batches: {batches}")

for i in range(0, batches):
    body = {
        "bulk": True,
        "items": []
    }
    
    i_start = i * batch_size
    i_end = len(df) if len(df) <= (batch_size * (i+1)) else (batch_size * (i+1))
    
    print(f'from {i_start} to {i_end}')
    timeseries_subset = df.iloc[i_start:i_end]
    timeseries_subset = timeseries_subset[['idSensore','Data','Valore']]
    timeseries_subset['Data'] = timeseries_subset['Data'].apply(arrange_timestamp)
    timeseries_subset = timeseries_subset.replace({np.nan: None})
    
    for i, sensor in timeseries_subset.iterrows():
        item = { "sensor_id_id": sensor.idSensore, "date": sensor.Data, "value": sensor.Valore }
        body["items"].append(item)
        
    body_json = json.dumps(body)
    body_json
    req = requests.post(base_url_measurements, body_json)
    if req.status_code == 500:
        print(req)
    print(f'request {i} - {req.status_code}')

Number of batches: 17291
from 0 to 1000
request 999 - 200
from 1000 to 2000
request 1999 - 200
from 2000 to 3000
request 2999 - 200
from 3000 to 4000
request 3999 - 200
from 4000 to 5000
request 4999 - 200
from 5000 to 6000
request 5999 - 200
from 6000 to 7000
request 6999 - 200
from 7000 to 8000
request 7999 - 200
from 8000 to 9000
request 8999 - 200
from 9000 to 10000
request 9999 - 200
from 10000 to 11000
request 10999 - 200
from 11000 to 12000
request 11999 - 200
from 12000 to 13000
request 12999 - 200
from 13000 to 14000
request 13999 - 200
from 14000 to 15000
request 14999 - 200
from 15000 to 16000
request 15999 - 200
from 16000 to 17000
request 16999 - 200
from 17000 to 18000
request 17999 - 200
from 18000 to 19000
request 18999 - 200
from 19000 to 20000
request 19999 - 200
from 20000 to 21000
request 20999 - 200
from 21000 to 22000
request 21999 - 200
from 22000 to 23000
request 22999 - 200
from 23000 to 24000
request 23999 - 200
from 24000 to 25000
request 24999 - 200
from 250

In [35]:
body = {
    "bulk": True,
    "items": []
}

timeseries_subset = df.iloc[0:1]
timeseries_subset = timeseries_subset[['idSensore','Data','Valore']]
timeseries_subset['Data'] = timeseries_subset['Data'].apply(arrange_timestamp)
timeseries_subset = timeseries_subset.replace({np.nan: None})

for i, sensor in timeseries_subset.iterrows():
    item = { "sensor_id": sensor.idSensore, "date": sensor.Data, "value": sensor.Valore }
    body["items"].append(item)
body_json = json.dumps(body)
body_json

'{"bulk": true, "items": [{"sensor_id_id": 5504, "date": "2023-01-01 00:00:00", "value": 42.1}]}'

In [55]:
count_elements = 2816286

size_b = 431521792
print(f'{size_b / count_elements} bytes per element in table')
print()

size_id = 22530288
print(f'{size_id / count_elements} bytes per element in column id')

size_date = 22530288
print(f'{size_date / count_elements} bytes per element in column date')

size_value = 18191588
print(f'{size_value / count_elements} bytes per element in column value')

size_fk = 11265144
print(f'{size_fk / count_elements} bytes per element in column fk')

print()
total_size = size_id + size_date + size_value + size_fk
print(f'{total_size} total size in bytes')
print(f'{total_size / count_elements} bytes per data element')
print(f'{total_size/1000000} MB')
print()

index_size = 83369984
print(f'{index_size / count_elements} bytes per element in index')

153.22371094412998 bytes per element in table

8.0 bytes per element in column id
8.0 bytes per element in column date
6.45942493056458 bytes per element in column value
4.0 bytes per element in column fk

74517308 total size in bytes
26.45942493056458 bytes per data element
74.517308 MB

29.60281164626036 bytes per element in index


In [52]:
total_data_AQ = 2800000 + 14800000 + 37900000 + 42900000
total_data_AQ

est_b_per_element = 154
est_in_db = total_data_AQ * 154
print(f'{est_in_db} bytes')
print(f'{est_in_db / 1000000} MB')
print(f'{est_in_db / 1000000000} GB')

15153600000 bytes
15153.6 MB
15.1536 GB


In [54]:
83369984

4278260.869565218